In [0]:
import numpy as np
from scipy.optimize import linprog
def nn2na(NN):
    # Get every location where exist an arc:
    idxs = np.argwhere(NN)
    # Preallocate NA matrix, dimension is (nodes, arcs)
    NA = np.zeros([NN.shape[0], idxs.shape[0]]).astype(int)
    C = np.zeros(NA.shape[1])
    # Loop in every arc, complete from (1) to (-1)
    for i, arc in enumerate(idxs):
        # Node arc:
        NA[arc[0], i] = 1 # From
        NA[arc[1], i] = -1 # To

    arc_idxs = [(arc[0], arc[1]) for arc in idxs]

    return NA, arc_idxs

# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

In [4]:
# IMPORT THE DATA:
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# DATA MANIPULATION:
C = np.array([2, 2, 2, 5, 2, 1, 2])
Aeq, arc_idxs = nn2na(NN)
beq = np.array([1, 0, 0, 0, 0, -1])
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n '
      'A_eq Node-Arc matrix:\n%s \n'
      'b_eq demand-supply vector: %s \n'
      'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds))

# OPTIMIZE:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds)

# GET THE SOLUTION:
selarcs = get_selected_arcs(arc_idxs, res.x)
print('## Results ##')
print('The raw solution will be: %s' % res.x)
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res.fun)

## Optimizer inputs ## 
Cost vector: [2 2 2 5 2 1 2] 
 A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]] 
b_eq demand-supply vector: [ 1  0  0  0  0 -1] 
Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 

## Results ##
The raw solution will be: [1.00000000e+00 1.07463814e-10 1.00000000e+00 4.87243456e-11
 1.07463814e-10 1.00000000e+00 1.07463814e-10]
The arcs that make the shortest path will be (from, to): [(0, 1), (1, 3), (3, 5)]
The minimum cost will be: 5.00 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
